In [0]:
%sql
SELECT
  run_id,
  period_start_time,
  sku_name,
  billing_origin_product,
  usage_date,
  SUM(usage_quantity) AS `DBUs`,
  elapsed_run_minutes
FROM
  system.billing.usage u
  join (select run_id, timestampdiff(MINUTE, period_start_time, period_end_time) as       elapsed_run_minutes, result_state, period_start_time,
        job_parameters['PROFILE']['value'] as run_name
        from system.lakeflow.job_run_timeline) jrl 
      on jrl.run_id  = u.usage_metadata.job_run_id
WHERE
  run_name in ('batch_aggr', 'batch_ingest')
  --and result_state = 'SUCCEEDED' 
GROUP BY ALL;